<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Decision Tree   
<font color=black size=4>
    Hamidreza Yaghoubi 
    
<br><br>
<font size=4>
In this homework, we are going to implement the Classification Decision Tree. Keep in mind to complete all of the following questions and write your own codes in the TODO cells.

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 2: Classification Decision Tree (100 points)
We will implement a Classification Decision Tree from scratch in the following problem. Then we will use our model to predict malignant and benign breast cancer. For this purpose, we will use the breast_cancer.csv dataset which you can find more details about it <a href="https://www.kaggle.com/datasets/merishnasuwal/breast-cancer-prediction-dataset"><font face="Roboto">here</font></a>.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import log
import random

<font face="Times New Roman" size=4><div dir=ltr>
## Classification Decision Tree Class (60 points)
In this section, you only need to fill TODO parts. You can find the logic and formula in both course slides and the web, but fill it out on your own. 

In [3]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        if self.value is not None:
            return True
        return False

In [4]:

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def is_splitting_finished(self, depth, num_class_labels, num_samples):
        if depth == self.max_depth:
            return True
        
        if num_samples <= self.min_samples_split:
            return True

        if num_class_labels == 1:
            return True

        return False

    def split(self, X, y, feature, threshold):
        left_indexes = X[feature] <= threshold
        right_indexes = -left_indexes
        X_left = X[left_indexes]
        y_left = y[left_indexes]
        X_right = X[right_indexes]
        y_right = y[right_indexes]

        return X_left, X_right, y_left, y_right

    def entropy(self, y):
        p = len(y[y.diagnosis == 1]) / len(y)
        print(len(y))
        if p == 1 or p == 0:
            return 0
        return -p * log(p, 2) - (1 - p) * log(1 - p, 2)


    def information_gain(self, X, y, feature, threshold):
        X_left, X_right, y_left, y_right = self.split(X, y, feature=feature, threshold=threshold)
        H_y = self.entropy(y=y)
        p = len(X[X[feature] >= threshold]) / len(X)

        if len(y_left) != 0:
            H_y_left = (1-p)*self.entropy(y=y_left)
        else:
            H_y_left = 0
        if len(y_right) != 0:
            H_y_right = (p)*self.entropy(y=y_right)
        else:
            H_y_right = 0
        return H_y - (H_y_right + H_y_left)

    def best_split(self, X, y):
        features = list(X.columns.values)
        random.shuffle(features)
        best_information_gain = 0
        best_feature = None
        best_threshold = None
        for feature in features:
            thresholds = list(set(list(X[feature])))
            for threshold in thresholds:
                info_gain = self.information_gain(X, y, feature, threshold)
                if info_gain >= best_information_gain:
                    best_information_gain = info_gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

    def build_tree(self, X, y, depth=0):
        if self.is_splitting_finished(depth, len(X.columns), len(X)):
            return None

        best_feature, best_threshold = self.best_split(X, y)
        X_left, X_right, y_left, y_right = self.split(X, y, best_feature, best_threshold)

        left_node = self.build_tree(X_left, y_left, depth=depth + 1)
        right_node = self.build_tree(X_right, y_right, depth=depth + 1)

        value = None
        if left_node is None or right_node is None:
            true_value = len(y[y['diagnosis'] == 1])
            false_value = len(y[y['diagnosis'] == 0])
            if true_value >= false_value:
                value = 1
            else:
                value = 0

        return Node(feature=best_feature, threshold=best_threshold, left=left_node, right=right_node, value=value)

    def fit(self, X, y):
        self.root = self.build_tree(X, y)

    def predict(self, X):
        tree = self.root
        predicted_value = []
        for index in list(X.index):
            data = X.loc[index]
            current_tree = tree
            for depth in range(self.max_depth):
                if Node.is_leaf(current_tree):
                    predicted_value.append(current_tree.value)
                    break
                feature = current_tree.feature
                threshold = current_tree.threshold
                if data[feature] <= threshold:
                    current_tree = current_tree.left
                if data[feature] > threshold:
                    current_tree = current_tree.right

        return predicted_value

<font face="Times New Roman" size=4><div dir=ltr>
## Data Prepration (20 points)
In this section, you must perform a good EDA for data. Then split it into train and validation data. We will then use the validation data to find the best model hyperparameters.  

In [5]:
breast_cancer_pdf = pd.read_csv("breast_cancer.csv", header=None)

In [6]:
column_names = ['mean_radius', 'mean_texture', 'mean_perimeter', 'mean_area', 'mean_smoothness', 'diagnosis']
breast_cancer_pdf.columns = column_names
breast_cancer_pdf.head(5)
breast_cancer_pdf.info()
breast_cancer_pdf.describe()

for column in column_names:
    print(breast_cancer_pdf[column].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   mean_radius      570 non-null    object
 1   mean_texture     570 non-null    object
 2   mean_perimeter   570 non-null    object
 3   mean_area        570 non-null    object
 4   mean_smoothness  570 non-null    object
 5   diagnosis        570 non-null    object
dtypes: object(6)
memory usage: 26.8+ KB
12.34    4
13.0     3
13.85    3
12.77    3
12.18    3
        ..
14.45    1
19.18    1
18.08    1
12.91    1
7.76     1
Name: mean_radius, Length: 457, dtype: int64
17.46    3
15.7     3
18.22    3
19.83    3
14.93    3
        ..
18.58    1
15.11    1
22.41    1
14.92    1
24.54    1
Name: mean_texture, Length: 480, dtype: int64
87.76    3
82.61    3
134.7    3
82.69    2
120.2    2
        ..
94.49    1
127.5    1
90.63    1
82.53    1
47.92    1
Name: mean_perimeter, Length: 523, dtype:

In [7]:
breast_cancer_pdf = pd.read_csv("breast_cancer.csv")

X = breast_cancer_pdf.drop(['diagnosis'], axis=1)

y = breast_cancer_pdf[['diagnosis']]
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.70, random_state=42)

<font face="Times New Roman" size=4><div dir=ltr>
## Training And Tuning Hyperparameters (20 points)
In this section, you only need to find the best hyperparameters for your model. You can test different values and permutations of hyperparameters by adding them to the lists below. Your model must have at least accuracy=0.85 on validation data.

In [10]:
max_depths = [2, 3, 4, 5]
min_samples_splits = [1, 2, 3]

In [11]:
best_max_depth = None
best_min_samples_split = None
best_accuracy = 0
best_model = None




for max_depth in max_depths:
    for min_samples_split in min_samples_splits:
        clf = DecisionTree(max_depth=max_depth, min_samples_split=min_samples_split)
        clf.fit(x_train, y_train)
        y_val_pred = clf.predict(x_val)
        y_train_pred = clf.predict(x_train)
        accuracy = accuracy_score(y_val_pred, y_val)
        train_accuracy = accuracy_score(y_train_pred, y_train)
        print(f"accuracy of training set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={train_accuracy}")
        print(f"accuracy of validation set for [min_samples_splits={min_samples_split}-max_depths={max_depth}] ={accuracy}")
        print("------------------------------------------------")
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_max_depth = max_depth
            best_min_samples_split = min_samples_split
            best_model = clf
            

170
1
169
170
3
167
170
4
166
170
13
157
170
6
164
170
28
142
170
20
150
170
48
122
170
61
109
170
85
85
170
74
96
170
87
83
170
82
88
170
83
87
170
124
46
170
145
25
170
152
18
170
157
13
170
121
49
170
95
75
170
138
32
170
136
34
170
132
38
170
147
23
170
166
4
170
158
12
170
162
8
170
168
2
170
169
1
170
170
170
109
61
170
112
58
170
8
162
170
17
153
170
27
143
170
26
144
170
24
146
170
34
136
170
32
138
170
29
141
170
42
128
170
55
115
170
50
120
170
51
119
170
54
116
170
52
118
170
62
108
170
70
100
170
64
106
170
69
101
170
56
114
170
71
99
170
73
97
170
75
95
170
77
93
170
89
81
170
103
67
170
91
79
170
104
66
170
94
76
170
90
80
170
105
65
170
107
63
170
113
57
170
88
82
170
102
68
170
119
51
170
118
52
170
126
44
170
123
47
170
122
48
170
134
36
170
131
39
170
133
37
170
135
35
170
93
77
170
140
30
170
101
69
170
141
29
170
130
40
170
153
17
170
161
9
170
160
10
170
163
7
170
164
6
170
167
3
170
79
91
170
2
168
170
9
161
170
11
159
170
7
163
170
18
152
170
25
145
170
31
139
17